In [1]:
!pip install transformers datasets torch tqdm

In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset
import torch
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))
model.to(device)

texts = [
    "Once upon a time in a distant galaxy, there lived a wise old robot.",
    "The jungle whispered secrets to the curious explorers.",
    "Technology is reshaping the future faster than ever before.",
    "The sky was filled with stars, each one telling a different story.",
    "Artificial intelligence will change the way we interact with machines."
]


dataset = Dataset.from_dict({"text": texts})


def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)


data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)


training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=50,
    prediction_loss_only=True,
    fp16=torch.cuda.is_available(),
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)


print("Training the model...")
trainer.train()


model.save_pretrained("./gpt2-finetuned")
tokenizer.save_pretrained("./gpt2-finetuned")
print("Model training complete and saved!")


model = GPT2LMHeadModel.from_pretrained("./gpt2-finetuned").to(device)
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2-finetuned")


def generate_text(prompt, max_new_tokens=300):  # Increase the token limit
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

    output = model.generate(
        input_ids,
        max_new_tokens=max_new_tokens,  
        do_sample=True,                
        top_k=50,                       
        top_p=0.95,                     
        temperature=0.8,                
        repetition_penalty=1.2,         
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id
    )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text



while True:
    prompt = input("Enter a prompt (or type 'exit' to quit): ")
    if prompt.strip().lower() == 'exit':
        print("Exiting the program.")
        break

    # Generate text based on the user's prompt
    generated_text = generate_text(prompt)
    print("\nGenerated Text:\n", generated_text)
    print("-" * 50)

Using device: cpu


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

C:\Users\SARANYA\AppData\Local\Temp\ipykernel_6156\3294954732.py:54: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Training the model...


C:\Users\SARANYA\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


...


Generated Text:
 Artificial intelligence will change the way we interact with machines and our brains.
 1) Machine learning could revolutionize how computers make decisions on behalf of humans 2)(iPad, iPad), 3). AI might enable a future that is far more accessible to people who are less constrained by technology's limitations (e-commerce giants like Amazon or Netflix in particular); it may also lead us towards smarter products for everyone; 4.) Machines learn from human actions through their own experiences rather than relying solely upon external systems which have learned its lessons over time 5). Artificial Intelligence can be used as an accelerator against existing problems – e.g., diseases such about Alzheimer's disease being solved using natural mechanisms [1]. In addition robots already play important roles within industries including medicine and health care due largely via robotic medical assistants provided they become better at recognizing certain stimuli faster/better kno

Enter a prompt (or type 'exit' to quit):  Story of robots in robot land



Generated Text:
 Story of robots in robot land
A new generation is rapidly developing a future where we can interact with machines to help us better understand our own behaviors, as well the world around them. This will be called robotics and it may turn out that humanity has already solved what's known about life on earth: how things work (or not) by building artificial intelligence hardware or software—the next step towards self-aware technology like AI for cars.


--------------------------------------------------


Enter a prompt (or type 'exit' to quit):  exit


Exiting the program.
